Лабораторная работа по ООП. Новокшанов Дмитрий.

Вариант:

In [1]:
((30 - 1) % 14) + 1

2

__Задача.__

Бакалавры с факультета математики открыли для себя множество комплексных чисел и теперь нуждаются в новом калькуляторе. Калькулятор должен поддерживать простейшие арифметические операции над числами, записанными в стеке.

2.1 Комплексное число: действительная и мнимая часть

2.2 Кватернион: действительная и три мнимых части (i, j, k)

2.3 Калькулятор (стек чисел): метод, принимающий арифметическую операцию, выполняющий её (по возможности) над доступными элементами стека и вставляющий результат этой операции на их место.

Link: https://ru.wikipedia.org/wiki/Кватернион


In [2]:
# other представляет собой объект того же класса, с которым произойдет операция.

class Quaternion:
    """ Класс кватернионов, принимающий действительную и три мнимых части числа.
        В нём определены стандартные арифметические операции между кватернионами. В виде строки число выводится в формате "a + bi + cj + dk"
    """
    def __init__(self, real, imaginary_i, imaginary_j, imaginary_k):
        self.real = real
        self.imaginary_i = imaginary_i
        self.imaginary_j = imaginary_j
        self.imaginary_k = imaginary_k

    # Переопределяем методы перегрузки операторов для кватерниона.
    def __add__(self, other):
        return Quaternion(self.real + other.real, self.imaginary_i + other.imaginary_i,
                          self.imaginary_j + other.imaginary_j, self.imaginary_k + other.imaginary_k)

    def __sub__(self, other):
        return Quaternion(self.real - other.real, self.imaginary_i - other.imaginary_i,
                          self.imaginary_j - other.imaginary_j, self.imaginary_k - other.imaginary_k)

    def __mul__(self, other):
        mul_quat_real = self.real * other.real - self.imaginary_i * other.imaginary_i - self.imaginary_j * other.imaginary_j - self.imaginary_k * other.imaginary_k
        mul_quat_im_i = self.real * other.imaginary_i + self.imaginary_i * other.real + self.imaginary_j * other.imaginary_k - self.imaginary_k * other.imaginary_j
        mul_quat_im_j = self.real * other.imaginary_j + other.real * self.imaginary_j + self.imaginary_k * other.imaginary_i - other.imaginary_k * self.imaginary_i
        mul_quat_im_k = self.real * other.imaginary_k + other.real * self.imaginary_k + self.imaginary_i * other.imaginary_j - other.imaginary_i * self.imaginary_j
        return Quaternion(mul_quat_real, mul_quat_im_i, mul_quat_im_j, mul_quat_im_k)

    def __truediv__(self, other):
        # Перевернуть other (сделать сопряженным), умножить это на self через __mul__ (*), поделить полученное на denominator (каждую часть)
        other_conjugate = Quaternion(other.real, -other.imaginary_i, -other.imaginary_j, -other.imaginary_k)
        q = Quaternion.__mul__(self, other_conjugate)
        denominator = other.real**2 + other.imaginary_i**2 + other.imaginary_j**2 + other.imaginary_k**2
        return Quaternion(q.real / denominator, q.imaginary_i / denominator, q.imaginary_j / denominator, q.imaginary_k / denominator)

    # Функция определяет, как объект будет представляться в виде строки при возвращении результата операции.
    def __str__(self):
        return f"{self.real} + {self.imaginary_i}i + {self.imaginary_j}j + {self.imaginary_k}k"

In [3]:
# Создаём класс комплексных чисел, который наследуется от класса кватернионов. Класс обязательно будет принимать действительную и одну мнимую части,
# а две другие мнимые части будут равняться нулям, что позволит не переопределять методы перегрузки операторов для орифметических операций с комплексными числами.
# Также это позволит свободно производить операции между кватернионами и комплексными часлами и наоборот.

class ComplexNumber(Quaternion):
    """ Класс комплексных чисел, принимающий действительную и мнимую части комплексного числа.
        Существует в виде кватерниона с мнимыми частями j и k равными нулю. Операции выполняются по схеме кватернионов. В виде строки число выводится в формате "a + bi"
    """
    def __init__(self, real, imaginary_i): # Функция инициализации объекта после его создания принимает действительную и мнимую части компексного числа.
        # Определяем принятые и унаследованные атрибуты аргумента.
        super().__init__(real, imaginary_i, 0, 0)

    # Переопределяем то, как объект будет представляться в виде строки при возвращении результата операции.
    def __str__(self):
        return f"{self.real} + {self.imaginary_i}i"

In [4]:
class Calculator:
    """ Класс калькулятора, производящий простые арифметические операции между объектами классов ComplexNumber, Quaternion и скалярными значениями классов int и float.
        Операции производятся над доступными элементами стека и результат этой операции вставляется на их место.
    """
    def __init__(self):
        self.stack = []

    def push(self, item):
        self.stack.append(item)

    def pop(self):
        if len(self.stack) < 1:
            return None
        return self.stack.pop()

    def perform_operation(self, operator):
        if len(self.stack) < 2:
            print("Недостаточно элементов в стаке.")
            return

        b = self.pop()
        a = self.pop()

        if operator == '+':
            if isinstance(a, int) and isinstance(b, int):
                result = a + b
            elif isinstance(a, float) and isinstance(b, float):
                result = a + b
            else:
                if isinstance(a, int) or isinstance(a, float):
                    a = ComplexNumber(a, 0)
                elif isinstance(b, int) or isinstance(b, float):
                    b = ComplexNumber(b, 0)
                result = a + b
                #Так как сами операции происходят с кватернионами, то переопределяем результат операции в класс комплексных чисел если j и k равны нулю.
                if result.imaginary_j == 0 and result.imaginary_k == 0:
                    result = ComplexNumber(result.real, result.imaginary_i)
        
        elif operator == '-':
            if isinstance(a, int) and isinstance(b, int):
                result = a - b
            elif isinstance(a, float) and isinstance(b, float):
                result = a - b
            else:
                if isinstance(a, int) or isinstance(a, float):
                    a = ComplexNumber(a, 0)
                elif isinstance(b, int) or isinstance(b, float):
                    b = ComplexNumber(b, 0)
                result = a - b
                if result.imaginary_j == 0 and result.imaginary_k == 0:
                    result = ComplexNumber(result.real, result.imaginary_i)

        elif operator == '*':
            if isinstance(a, int) and isinstance(b, int):
                result = a * b
            elif isinstance(a, float) and isinstance(b, float):
                result = a * b
            else:
                if isinstance(a, int) or isinstance(a, float):
                    a = ComplexNumber(a, 0)
                elif isinstance(b, int) or isinstance(b, float):
                    b = ComplexNumber(b, 0)
                result = a * b
                if result.imaginary_j == 0 and result.imaginary_k == 0:
                    result = ComplexNumber(result.real, result.imaginary_i)

        elif operator == '/':
            if b == 0:
                print('Деление на ноль')
            else:
                if isinstance(a, int) and isinstance(b, int):
                    result = a / b
                elif isinstance(a, float) and isinstance(b, float):
                    result = a / b
                else:
                    if isinstance(a, int) or isinstance(a, float):
                        a = ComplexNumber(a, 0)
                    elif isinstance(b, int) or isinstance(b, float):
                        b = ComplexNumber(b, 0)
                    result = a / b
                    if result.imaginary_j == 0 and result.imaginary_k == 0:
                        result = ComplexNumber(result.real, result.imaginary_i)

        self.push(result)
        print(f"Ответ: {result}")

Проверяем работу калькулятора со скалярными значениями.
(Проверял в том числе тут: https://www.redcrab-software.com/en/Calculator/Quaternion-Calculator)

In [5]:
calc = Calculator()
num1 = 1
num2 = 2
calc.push(num1)
calc.push(num2)
calc.perform_operation('+')
print(type(calc.pop()))

calc.push(num1)
calc.push(num2)
calc.perform_operation('-')

calc.push(num1)
calc.push(num2)
calc.perform_operation('*')

calc.push(num1)
calc.push(num2)
calc.perform_operation('/')
print(type(calc.pop()))



Ответ: 3
<class 'int'>
Ответ: -1
Ответ: 2
Ответ: 0.5
<class 'float'>


Проверяем работу калькулятора с комплексными числами.

In [6]:
cnum1 = ComplexNumber(1, 2)
cnum2 = ComplexNumber(3, 4)
calc.push(cnum1)
calc.push(cnum2)
calc.perform_operation('+')
print(type(calc.pop()))

calc.push(cnum1)
calc.push(cnum2)
calc.perform_operation('-')

calc.push(cnum1)
calc.push(cnum2)
calc.perform_operation('*')

calc.push(cnum1)
calc.push(cnum2)
calc.perform_operation('/')
print(type(calc.pop()))

Ответ: 4 + 6i
<class '__main__.ComplexNumber'>
Ответ: -2 + -2i
Ответ: -5 + 10i
Ответ: 0.44 + 0.08i
<class '__main__.ComplexNumber'>


Проверяем арифметические операции между комплексными числами и скалярными значениями в обе стороны.

In [7]:
calc.push(num1)
calc.push(cnum2)
calc.perform_operation('+')
print(type(calc.pop()))

calc.push(cnum2)
calc.push(num1)
calc.perform_operation('-')

calc.push(num1)
calc.push(cnum2)
calc.perform_operation('*')

calc.push(cnum2)
calc.push(num1)
calc.perform_operation('/')
print(type(calc.pop()))

Ответ: 4 + 4i
<class '__main__.ComplexNumber'>
Ответ: 2 + 4i
Ответ: 3 + 4i
Ответ: 3.0 + 4.0i
<class '__main__.ComplexNumber'>


Проверяем работу калькулятора кватернионами.

In [10]:
quat1 = Quaternion(1, 2, 3, 4)
quat2 = Quaternion(2, 3, 4, 5)
calc.push(quat1)
calc.push(quat2)
calc.perform_operation('+')
print(type(calc.pop()))

calc.push(quat1)
calc.push(quat2)
calc.perform_operation('-')

calc.push(quat1)
calc.push(quat2)
calc.perform_operation('*')

calc.push(quat1)
calc.push(quat2)
calc.perform_operation('/')
print(type(calc.pop()))

Ответ: 3 + 5i + 7j + 9k
<class '__main__.Quaternion'>
Ответ: -1 + -1i + -1j + -1k
Ответ: -36 + 6i + 12j + 12k
Ответ: 0.7407407407407407 + 0.037037037037037035i + 0.0j + 0.07407407407407407k
<class '__main__.Quaternion'>


Проверяем работу арифметические операции между кватернионами и скалярными значениями в обе стороны.

In [11]:
calc.push(num2)
calc.push(quat1)
calc.perform_operation('+')
print(type(calc.pop()))

calc.push(quat1)
calc.push(num2)
calc.perform_operation('-')

calc.push(num2)
calc.push(quat1)
calc.perform_operation('*')

calc.push(quat1)
calc.push(num2)
calc.perform_operation('/')
print(type(calc.pop()))

Ответ: 3 + 2i + 3j + 4k
<class '__main__.Quaternion'>
Ответ: -1 + 2i + 3j + 4k
Ответ: 2 + 4i + 6j + 8k
Ответ: 0.5 + 1.0i + 1.5j + 2.0k
<class '__main__.Quaternion'>


Проверяем работу арифметических операций между кватернионами комплексными числами в обе стороны.

In [12]:
calc.push(cnum1)
calc.push(quat1)
calc.perform_operation('+')
print(type(calc.pop()))

calc.push(quat1)
calc.push(cnum1)
calc.perform_operation('-')

calc.push(cnum1)
calc.push(quat1)
calc.perform_operation('*')

calc.push(quat1)
calc.push(cnum1)
calc.perform_operation('/')
print(type(calc.pop()))

Ответ: 2 + 4i + 3j + 4k
<class '__main__.Quaternion'>
Ответ: 0 + 0i + 3j + 4k
Ответ: -3 + 4i + -5j + 10k
Ответ: 1.0 + 0.0i + -1.0j + 2.0k
<class '__main__.Quaternion'>
